<a href="https://colab.research.google.com/github/boraturant/Word2Vec_Instacart_Similarity_Complementarity/blob/master/X2vec_Instacart_Gensim_FINAL_FOR_GITHUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sklearn
!pip install --upgrade gensim

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import gensim

%load_ext tensorboard
%tensorflow_version 2.x
import tensorflow as tf

from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence

import io
import urllib
import collections
import os
import datetime
import zipfile
import urllib.request

from sklearn.model_selection import train_test_split# Load the TensorBoard notebook extension


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
#DOWNLOAD CSVs
#CSVs must be uploaded to your Google drive
#3 million grocery orders from more than 200,000 Instacart users

orders = pd.read_csv("/content/drive/My Drive/Instacart_Dataset/orders.csv")
prior_orders = pd.read_csv("/content/drive/My Drive/Instacart_Dataset/order_products__prior.csv")
train_orders = pd.read_csv("/content/drive/My Drive/Instacart_Dataset/order_products__train.csv")
products = pd.read_csv("/content/drive/My Drive/Instacart_Dataset/products.csv").set_index('product_id')
departments = pd.read_csv("/content/drive/My Drive/Instacart_Dataset/departments.csv")

In [0]:
#CONVERT COLUMN TO STRING

prior_orders["product_id"] = prior_orders["product_id"].astype(str) 
train_orders["product_id"] = train_orders["product_id"].astype(str)  

In [0]:
#GET SENTENCES

prior_products = prior_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())
train_products = train_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())

In [0]:
#GET SENTENCES

#sentences = prior_products.append(train_products)
sentences=prior_products

#NOTE: No need to keep all sentences in memory, could be loaded by iterators (https://rare-technologies.com/word2vec-tutorial/)

longest = np.max(sentences.apply(len))
#print(longest)

In [0]:
# CONVERT TO NUMPY ARRAY
sentences = sentences.values

In [0]:
#THE MODEL

window_size=longest
epochs=5
embedding_size=100
min_count=5  #Drop infrequent items in dataset
number_of_negative_samples=7

ns_exponent = 0

sample=0 #Not included
#(float, optional) – The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).

#RUN THE MODEL
model = gensim.models.Word2Vec(sentences, sg=1, size=embedding_size, window=window_size, min_count=min_count, workers=4, hs=0, negative=number_of_negative_samples,ns_exponent=ns_exponent, iter=epochs)


In [0]:
#SAVE MODEL

model.save('/content/mymodel')

In [0]:
#LOAD MODEL
#model = gensim.models.Word2Vec.load('/content/mymodel')

In [0]:
'''
wv
This object essentially contains the mapping between words and embeddings. After training, it can be used directly to query those embeddings in various ways. 
See the module level docstring for examples.
Type:	Word2VecKeyedVectors

vocabulary
This object represents the vocabulary (sometimes called Dictionary in gensim) of the model. Besides keeping track of all unique words, 
this object provides extra functionality, such as constructing a huffman tree (frequent words are closer to the root), or discarding extremely rare words.
Type:	Word2VecVocab

trainables
This object represents the inner shallow neural network used to train the embeddings. The semantics of the network differ slightly in the 
two available training modes (CBOW or SG) but you can think of it as a NN with a single projection and hidden layer which we train on the corpus. 
The weights are then used as our embeddings (which means that the size of the hidden layer is equal to the number of features self.size).
Type:	Word2VecTrainables
'''

In [0]:
#The trained word vectors are stored in a KeyedVectors instance in model.wv:
#vector = model.wv['computer']  # numpy vector of a word

#You can perform various NLP word tasks with a trained model. Some of them are already built-in - you can see it in gensim.models.keyedvectors.

#word_vectors = model.wv
#del model

In [0]:
#NORMALIZE VECTORS

#If you explicitly do an init_sims(replace=True) call,
#you'll actually clobber the raw vectors, in-place, with the unit-normed vectors.

#Precompute L2-normalized vectors.
model.init_sims(replace=True) #Normalize Vectors

#Deprecated. Use self.wv.init_sims instead. See init_sims().

In [0]:
#This is the target matrix (skip-gram negative sampling)
#model.wv.vectors.shape 
#model.wv.vectors[1]

#model.wv.vocab

#model.wv.init_sims(replace=True)
#model.wv.vectors_norm.shape  #(Normalized)
#model.wv.vectors_norm[1]

#This is the context matrix (skip-gram negative sampling)
#model.trainables.syn1neg.shape

In [0]:
#CREATE EMBEDDING FILES FOR VISUALIZATION

vocab_size=len(model.wv.vocab)

my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    #my_dict[key] = model.wv[key]
    my_dict[key] = model.wv.get_vector(key)
    #my_dict[key] = model.wv.word_vec(key, use_norm=True)

#File Names
target_vecs_fileLoc='target_vecs_GensimFinal.tsv'
target_meta_fileLoc='target_meta_GensimFinal.tsv'


#Delete if exists
if os.path.exists(target_vecs_fileLoc):
  os.remove(target_vecs_fileLoc)
  
if os.path.exists(target_meta_fileLoc):
  os.remove(target_meta_fileLoc)
  
out_v = io.open(target_vecs_fileLoc, 'w', encoding='utf-8')
out_m = io.open(target_meta_fileLoc, 'w', encoding='utf-8')

#Meta File Header
#out_m.write('ID\tProductName\tDepartment\tEmbedding' + "\n")
out_m.write('ID\tProductName\tDepartment' + "\n")

for word_index in range(vocab_size):
  
  word =  list(model.wv.vocab.keys()) [word_index] 
    
  #Embedding Vector
  embeddings = model.wv.get_vector(word)
  #embeddings = my_dict[word]

  #META Input
  productname = products.loc[int(word),'product_name']   
  departmentName= str(products.loc[int(word),'department_id']) #str(departments[departments.department_id==int(products.loc[int(word),'department_id'])].department.apply(str)[0])
 
  wordFinal= word + '\t' + productname + '\t'+departmentName #+ '\t' + ','.join([str(x) for x in embeddings]) 
  
  out_m.write(wordFinal + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()




In [0]:
# https://projector.tensorflow.org/
# Save File and open in Embedding Projector
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(target_vecs_fileLoc)
  files.download(target_meta_fileLoc)

# **Similarity**

In [0]:
def _l2_norm(m, replace=False):
    """Return an L2-normalized version of a matrix.
    Parameters
    ----------
    m : np.array
        The matrix to normalize.
    replace : boolean, optional
        If True, modifies the existing matrix.
    Returns
    -------
    The normalized matrix.  If replace=True, this will be the same as m.
    """
    dist = np.sqrt((m ** 2).sum(-1))[..., np.newaxis]
    if replace:
        m /= dist
        return m
    else:
        return (m / dist).astype(np.float32)

In [0]:
def Get_Similar_Items_With_Vector(target_Vector, number_of_items=10):

  target_item_vector=_l2_norm(target_Vector,replace=False)
  
  vectors=[]
  
  #Source vectors must be in Normalized form
  vectors=_l2_norm(model.wv.vectors,replace=False)
 
  #DOT PRODUCT (COSINE SIMILARITY)  -1 0 1
  dists = np.dot(vectors, target_item_vector)
  
  #sorted_dists= np.argsort(dists) #SORT - Sort distance indexes from smallest to largest!
  #sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items, reverse=False) # Same as the above line
  
  #IMPORTANT! sorted_dists must be from largest to smallest
  sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items+1, reverse=True) # Same as the above line
 
  # Take the last n sorted distances
  closest = sorted_dists[:number_of_items+1]
  #return result[:topn]
  #closest = closest[::-1]

  #Don't return the searched item
  result = [(model.wv.index2word[sim], float(dists[sim])) for sim in closest ]
  return result[:number_of_items]
    
  #RETURN ITEM IDs
  #result=[]
  #for i in closest:
  #  result.append(model.wv.index2word[i])

  #return result

In [0]:
def Get_Similar_Items_IDs(target_Item, number_of_items=10):
  
  #target_item_vector = model.wv.word_vec(target_Item, use_norm=True) 
  target_item_vector= model.wv.get_vector(target_Item)
  
  target_item_vector=_l2_norm(target_item_vector,replace=False)
  
  vectors=[]
  
  #Source vectors must be in Normalized form
  vectors=_l2_norm(model.wv.vectors,replace=False)
 
  #DOT PRODUCT (COSINE SIMILARITY)  -1 0 1
  dists = np.dot(vectors, target_item_vector)
  
  #sorted_dists= np.argsort(dists) #SORT - Sort distance indexes from smallest to largest!
  #sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items, reverse=False) # Same as the above line
  
  #IMPORTANT! sorted_dists must be from largest to smallest
  sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items+1, reverse=True) # Same as the above line
 
  # Take the last n sorted distances
  closest = sorted_dists[:number_of_items+1]
  #return result[:topn]
  #closest = closest[::-1]

  #Don't return the searched item
  result = [(model.wv.index2word[sim], float(dists[sim])) for sim in closest if sim not in [model.wv.vocab[target_Item].index]]
  return result[:number_of_items]
    
  #RETURN ITEM IDs
  #result=[]
  #for i in closest:
  #  result.append(model.wv.index2word[i])

  #return result
  
  

In [0]:
def Get_Complementary_Items_With_Vector(target_Vector, number_of_items=10, useNorm=False):
  
  if useNorm:
      target_item_vector = _l2_norm(target_Vector,replace=False) 
  else:
      target_item_vector = target_Vector
  
  #target_item_vector= model.wv.get_vector(target_Item)
  
  vectors=[]
  
  #Source vectors must not be normalized
  if useNorm:
      vectors= _l2_norm(model.trainables.syn1neg,replace=False)  
  else:
      vectors=model.trainables.syn1neg 
   
  
  #DOT PRODUCT (NOT COSINE SIMILARITY)  
  dists = np.dot(vectors, target_item_vector)
  
  #sorted_dists= np.argsort(dists) #SORT - Sort distance indexes from smallest to largest!
  #sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items, reverse=False) # Same as the above line
  
  #IMPORTANT! sorted_dists must be from largest to smallest
  sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items+1, reverse=True) # Same as the above line
 
  # Take the last n sorted distances
  closest = sorted_dists[:number_of_items+1]
  #return result[:topn]
  #closest = closest[::-1]

  #Don't return the searched item
  result = [(model.wv.index2word[sim], float(dists[sim])) for sim in closest]
  return result[:number_of_items]
    
  #RETURN ITEM IDs
  #result=[]
  #for i in closest:
  #  result.append(model.wv.index2word[i])

  #return result

In [0]:
def Get_Complementary_Items_IDs(target_Item, number_of_items=10, useNorm=False):
  
  #Do not use norm vectors
  #Similarity uses DotProduct, do not use norm vectors
  
  if useNorm:
      target_item_vector = _l2_norm(model.wv.word_vec(target_Item, use_norm=False),replace=False) 
  else:
      target_item_vector = model.wv.word_vec(target_Item, use_norm=False) 
  
  
  #target_item_vector= model.wv.get_vector(target_Item)
  
  vectors=[]
  
  #Source vectors must not be normalized
  if useNorm:
      vectors= _l2_norm(model.trainables.syn1neg,replace=False)  
  else:
      vectors=model.trainables.syn1neg
   
  
  #DOT PRODUCT (NOT COSINE SIMILARITY)  
  dists = np.dot(vectors, target_item_vector)
  
  #sorted_dists= np.argsort(dists) #SORT - Sort distance indexes from smallest to largest!
  #sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items, reverse=False) # Same as the above line
  
  #IMPORTANT! sorted_dists must be from largest to smallest
  sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items+1, reverse=True) # Same as the above line
 
  # Take the last n sorted distances
  closest = sorted_dists[:number_of_items+1]
  #return result[:topn]
  #closest = closest[::-1]

  #Don't return the searched item
  result = [(model.wv.index2word[sim], float(dists[sim])) for sim in closest if sim not in [model.wv.vocab[target_Item].index]]
  return result[:number_of_items]
    
  #RETURN ITEM IDs
  #result=[]
  #for i in closest:
  #  result.append(model.wv.index2word[i])

  #return result
  

In [0]:
def Get_Similar_or_Complementary_Items_IDs(target_Item, similar, number_of_items=10):
  
  #Get Normalized Target Vector
  #target_item_vector = model.wv.word_vec(target_Item, use_norm=True) 
  target_item_vector= model.wv.get_vector(target_Item)
  
  vectors=[]
  
  #Source vectors must be in Normalized form
  if similar:
    vectors=model.wv.vectors
  else:
    #Full Context Matrix
    #model.trainables.syn1neg.shape
    #fullContextVectors=model.wv.vectors #model.trainables.syn1neg
    vectors=_l2_norm(model.trainables.syn1neg,False)
    
  #DOT PRODUCT (COSINE SIMILARITY)  -1 0 1
  dists = np.dot(vectors, target_item_vector)
  
  #SORT - Sort distance indexes from smallest to largest!
  sorted_dists= np.argsort(dists)
  #sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items, reverse=False) # Same as the above line
  
  #IMPORTANT! sorted_dists must be from largest to smallest
  sorted_dists = gensim.matutils.argsort(dists, topn=number_of_items+1, reverse=True) # Same as the above line
 
  # Take the last n sorted distances
  closest = sorted_dists[:number_of_items+1]
  #return result[:topn]
  #closest = closest[::-1]

  #Don't return the searched item
  result = [(model.wv.index2word[sim], float(dists[sim])) for sim in closest if sim not in [model.wv.vocab[target_Item].index]]
  return result[:number_of_items]
    
  #RETURN ITEM IDs
  #result=[]
  #for i in closest:
  #  result.append(model.wv.index2word[i])

  #return result

In [0]:
def Get_Complementary_Items_IDs_WithGemsim(target_Item, embeddingDimension=100, number_of_items=10):

  outv = gensim.models.KeyedVectors(embeddingDimension)
  outv.vocab = model.wv.vocab  # same
  outv.index2word = model.wv.index2word  # same
  outv.syn0 = model.syn1neg  # different
  #inout_similars = outv.most_similar('9065', topn=10)

  inout_similars = outv.similar_by_vector(model.wv.get_vector(target_Item), topn=number_of_items) 
  #inout_similars = outv.most_similar(target_Item, topn=number_of_items) 
  
  return inout_similars

# **Evaluation**

In [0]:
#Load Model if trained before

#model_File='/content/drive/My Drive/Models/mymodel_EP10_NEG20_NS00'
#model = gensim.models.Word2Vec.load(model_File)

In [0]:
# Test set
train_orders = pd.read_csv("/content/drive/My Drive/Instacart_Dataset/order_products__train.csv")
train_orders["product_id"] = train_orders["product_id"].astype(str)  
train_products = train_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())


In [0]:
#In basket - Next product recommendation
order_count=0
evaluated_order_count=0

evaluate_min_item_orders=2
evaluate_n_missing_items=1
hitCount=0

import time
start_time = time.time()

#Loop Orders
for items in train_products.iteritems(): 
    order_count += 1
    
    #OrderID and Products
    order_ID=items[0]
    product_IDs=items[1]
    
    #Evaluate min basket size
    if len(product_IDs)<=evaluate_min_item_orders:
      continue
    
    #Pop n items from product_IDs
    removed_ProductIDs=[]
    for i in range(evaluate_n_missing_items):
      removed_ProductIDs.append(product_IDs.pop())
      
      
    #Get Vectors for Product IDs
    try:
      product_ID_Vectors=[_l2_norm(model.wv.get_vector(product_ID),replace=False) for product_ID in product_IDs]
    except:
      continue
      
    evaluated_order_count += 1
    
    product_ID_Vectors=np.asarray(product_ID_Vectors)
    
    #Get averaged Product_ID_Vectors
    averaged_vector_for_products= np.mean(product_ID_Vectors,0)
    #averaged_vector_for_products=tf.reduce_mean(product_ID_Vectors,0)
    #print(averaged_vector_for_products)
    
    #Get Recommendations for the averaged_vector_for_products
    recommendations=Get_Complementary_Items_With_Vector(averaged_vector_for_products,10,useNorm=True)
    #recommendations=Get_Similar_Items_With_Vector(averaged_vector_for_products,10)
    recommendations=[ recommendation[0] for recommendation in recommendations]
    #print(recommendations)
    
    #print(type(product_IDs))
    #print(len(product_IDs))
    
    
    if (removed_ProductIDs[0] in recommendations) :
      hitCount+=1
    
    #if evaluated_order_count==60000:
    #  break

print (hitCount/evaluated_order_count*100)

0.8066596721762612


In [0]:
print("--- %s seconds ---" % (time.time() - start_time))